In [710]:
# requirments
# !pip install pandas numpy pandas_datareader

In [711]:
import os
from dateutil import relativedelta, rrule
from datetime import datetime, date, timedelta
import functools
import math

import pandas as pd
from pandas import Series, DataFrame
import pandas_datareader.data as web

In [712]:
# paths
PATH_DATA = os.path.join(
    "data"
)
PATH_EXPORTS = os.path.join(
    "exports"
)
PATH_SOURCE_CSV = os.path.join(
    PATH_DATA,
    "cleaned_data_export.csv"
)
PATH_EXPORT_DAILY_RET = os.path.join(
    PATH_EXPORTS,
    "daily_returns.csv"
)
PATH_EXPORT_MONTHLY_RET = os.path.join(
    PATH_EXPORTS,
    "monthly_returns.csv"
)
PATH_EXPORT_FORMULATION_RET = os.path.join(
    PATH_EXPORTS,
    "formulation_returns.csv"
)
PATH_EXPORT_HOLDING_WINNERS = os.path.join(
    PATH_EXPORTS,
    "holding_winners.csv"
)
PATH_EXPORT_HOLDING_LOOSERS = os.path.join(
    PATH_EXPORTS,
    "holding_loosers.csv"
)
PATH_EXPORT_HOLDING_RET = os.path.join(
    PATH_EXPORTS,
    "holding_return.csv"
)
PATH_EXPORT_RANKS = os.path.join(
    PATH_EXPORTS,
    "ranks.csv"
)

# dates
DATE_SIMULATION_START = datetime(1998, 1, 1)
DATE_SIMULATION_END = datetime(1999, 12, 1)
DATE_CORONA = datetime(2020, 3, 1)
WEIGHT_READJUST_DATES = [
   DATE_CORONA
]

# const vars
INTERVAL_FORMULATION = 12 # months
INTERVAL_HOLDING = 3 # months
TOP_N_PERCENT = 0.5
BOTTOM_N_PERCENT = 0.5
TICKER_FILTER_LIST = ["CERG", "JEF", "ALJJ", "APF", "PIY", "KINS", "ROCM", "PETS", "HHY", "FFCO"]
ENABLE_TICKER_FILTER = True
COLUMNS_MONTHLY_HOLDING_RETURNS = ["year", "month", "winner_returns", "looser_returns", "winner_ticker", "looser_ticker"]

# init vars
INIT_STOCK_SUMMARY = {
    "stock_count": 0,
    "current_value": 0
}

In [713]:
def request_data(ticker, request_start_date, request_end_date, download=False):
    # init df
    res_df = None

    # request datareader
    try:
        res_df = web.DataReader(ticker, 'yahoo', request_start_date, request_end_date)

        #
        if download:
            res_df.to_csv("data/" + ticker + "-" + str(request_start_date) + "-" + request_end_date + ".csv", index=False)
    except:
        print("Error:", ticker, request_start_date, request_end_date)

    return res_df    

def get_market_daily_closing(ticker, daily_date_start):
    #
    daily_returns = 0

    #
    daily_date_end = daily_date_start + timedelta(days=1)

    # request data
    market_daily_stats_df = request_data(ticker, daily_date_start, daily_date_end)

    if market_daily_stats_df is not None:
        # R(i,t)
        daily_returns = 1 + (market_daily_stats_df['Close'] - market_daily_stats_df['Open'])
    
    #
    return daily_returns

def get_market_monthly_closing(ticker, start_date, end_date):
    #
    stock_returns_monthly = 0

    for month_start_date in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        #
        month_end_date = month_start_date + relativedelta.relativedelta(months=1, day=1)

        #
        stock_returns_daily = 0
        
        for daily_date in rrule.rrule(rrule.DAILY, dtstart=month_start_date, until=month_end_date):
            #
            stock_returns_daily += get_market_daily_closing(ticker, daily_date)

        #
        stock_returns_monthly += stock_returns_daily

        print("returns monthly:" , stock_returns_monthly, "\n")
            
# 
def get_market_stats_from_internet(ticker_list, start_date, end_date):
    #
    market_stats = dict()

    #
    for ticker in ticker_list:
        #
        get_market_monthly_closing(ticker, start_date, end_date)

    return market_stats

In [714]:
def read_data(path=PATH_SOURCE_CSV, rows=1000000):
    if ENABLE_TICKER_FILTER:
        return pd.read_csv(path, nrows=rows)    

    return pd.read_csv(path)    

def preprocess_data(data_df):
    # format dt
    data_df['date'] = pd.to_datetime(data_df['datadate'])

    # sort
    data_df.sort_values(by=['tic', 'date'], inplace=True)

    # set index
    # data_df.index = data_df['date']
    
    # extract month year
    data_df['year'] = data_df['date'].dt.year
    data_df['month'] = data_df['date'].dt.month
    data_df['month_startdate'] = data_df['date'].to_numpy().astype('datetime64[M]')

    # use fukter for debugging
    if ENABLE_TICKER_FILTER:
        data_df = data_df.loc[data_df['tic'].isin(TICKER_FILTER_LIST)]

    return data_df   

def init_stock_summary_dict(ticker):
    return dict({ticker: INIT_STOCK_SUMMARY})
# 
def get_portfolio_dict(market_revenue_stats):
    #
    stock_list = market_revenue_stats.tic.unique()

    #
    init_portfolio_dict = list(map(init_stock_summary_dict, stock_list))

    return init_portfolio_dict, stock_list

def get_iteration_intervals(start_date, end_date, interval_months=INTERVAL_FORMULATION):
    #
    intervals = []

    #
    interval_ticks = pd.date_range(start=start_date, end=end_date, freq=pd.offsets.MonthEnd(interval_months))

    #
    for idx, tick in enumerate(interval_ticks):
        if idx > len(interval_ticks) - 2:
            continue

        intervals.append((
            interval_ticks[idx],
            interval_ticks[idx+1]   
        ))

    return intervals

def get_aggregate_month_plus_one(revenues):    
    revenue_pro = None
    for revenue in revenues:
        if revenue > 0:
            if revenue_pro is None:
                revenue_pro = (1 + revenue) 
            else:
                revenue_pro *= (1 + revenue)

    if revenue_pro is None:
        return 0
        
    return revenue_pro - 1

def get_aggregate_month(revenues):    
    revenue_pro = None
    for revenue in revenues:
        if revenue > 0:
            if revenue_pro is None:
                revenue_pro = revenue
            else:
                revenue_pro *= revenue

    if revenue_pro is None:
        return 0
        
    return revenue_pro - 1

def get_avg_portfolio_return(portfolio_returns):
    return portfolio_returns.mean()

def calculate_monthly_returns(raw_data_df):
    processed_df = raw_data_df.copy()
    processed_df["monthly_return"] = raw_data_df.groupby(["tic", "month_startdate"]).daily_return.transform(get_aggregate_month)
    processed_df = processed_df.drop_duplicates(subset=["tic", 'month_startdate'])
    # processed_df.index = processed_df["month_startdate"]
    
    return processed_df

def calculate_formulation_returns(raw_data_df):
    processed_df = raw_data_df.copy()
    formulation_returns = calculate_returns(processed_df, INTERVAL_FORMULATION,  "tic", "monthly_return", "formulation_returns")

    return formulation_returns
    
def calculate_holding_returns(raw_data_df):
    processed_df = raw_data_df.copy()
    holding_returns = calculate_returns(processed_df, INTERVAL_HOLDING,  "tic", "monthly_return", "holding_returns")

    return holding_returns

def calculate_returns(raw_data_df, period, group_columns, apply_column, new_column_name):
    monthly_grouped_df = raw_data_df.copy()
    monthly_grouped_df[new_column_name] = monthly_grouped_df.groupby(group_columns)[apply_column].transform(lambda s: s.rolling(period).apply(get_aggregate_month_plus_one))

    return monthly_grouped_df

def calculate_avg_holding_returns(raw_data_df, period, group_columns, apply_column, new_column_name):
    avg_holding_df = raw_data_df.copy()

    avg_holding_df[new_column_name] = avg_holding_df.groupby(group_columns)[apply_column].transform(lambda s: s.rolling(period).apply(get_avg_portfolio_return))

    print("calc")
    return avg_holding_df

def get_holding_avg_returns(raw_data_df, tic_list, month_startdate):
    processed_df = raw_data_df.copy()
    year, month = month_startdate.year, month_startdate.month

    # put tic, year, month filter
    filtered_df = processed_df[(processed_df['month'] >= month) & (processed_df['month'] < month+INTERVAL_HOLDING) & (processed_df['year'] == year)]
    filtered_df = filtered_df[filtered_df['tic'].isin(tic_list)] 

    #
    holding_df = calculate_avg_holding_returns(filtered_df, INTERVAL_HOLDING,  "tic", "monthly_return", "holding_returns")

    return holding_df.holding_returns.mean()

def get_winner_looser_list(raw_data_df, month_startdate, winner_looser_type):
    processed_df = raw_data_df.copy()
    year, month = month_startdate.year, month_startdate.month
    
    #
    filtered_df = processed_df[(processed_df['month'] == month) & (processed_df['year'] == year)]

    #
    tic_list = filtered_df[winner_looser_type]

    if not tic_list.empty:
        return tic_list.unique()[0].split(",")

    return []

#
def check_weight_readjustment_date(budget_perc_1, budget_perc_2):
    pass

#
def get_short_long_call(stock_stats_dict):
    # if last 12 month max greater than current price, decide short or long
    # get_market_stats()
    pass

def get_winner_losers(unlabelled_df, n_ranks, label):
    # get
    filtered_ranks = unlabelled_df.loc[unlabelled_df['compiled_rank'].isin(n_ranks)]
    
    return label

def get_ranking_by_interval(raw_data_df):
    processed_df = raw_data_df.copy()

    # get ranks
    processed_df["compiled_rank"] = processed_df.groupby(["month_startdate"]).formulation_returns.transform(pd.DataFrame.rank, ascending=False)
    processed_df = processed_df.sort_values(["year", "month", "compiled_rank"], ascending=True)

    # drop nans
    processed_df = processed_df[processed_df['compiled_rank'].notna()]

    # get top and bottom n rank ranges
    ranks_range = processed_df["compiled_rank"].unique() 
    ranks_count = len(ranks_range)
    top_n_ranks = ranks_range[:math.ceil(ranks_count * TOP_N_PERCENT)]
    bottom_n_ranks = ranks_range[-math.ceil(ranks_count * BOTTOM_N_PERCENT):]
    
    # assign winner/looser label
    processed_df.loc[processed_df.compiled_rank.isin(top_n_ranks), 'status'] = 'w'
    processed_df.loc[processed_df.compiled_rank.isin(bottom_n_ranks), 'status'] = 'l'    

    #
    winner_df = processed_df.loc[processed_df['status'] == 'w']
    winner_df["winners"] = winner_df.groupby(["month_startdate"]).tic.transform(lambda x: ",".join(x.astype(str)))
    # winner_df = winner_df.drop_duplicates(subset=['month_startdate'])

    looser_df = processed_df.loc[processed_df['status'] == 'l']
    looser_df["loosers"] = looser_df.groupby(["month_startdate"]).tic.transform(lambda x: ",".join(x.astype(str)))
    # looser_df = looser_df.drop_duplicates(subset=['month_startdate'])

    return processed_df, winner_df, looser_df

In [715]:
#
def run_simulation(start_date, end_date):
    # read
    data_df = read_data()

    # preprocess data
    data_df = preprocess_data(data_df)
    data_df.to_csv(PATH_EXPORT_DAILY_RET)

    # calculate monthly returns
    monthly_df = calculate_monthly_returns(data_df)
    monthly_df.to_csv(PATH_EXPORT_MONTHLY_RET)

    # calculate formulation returns
    formulation_df = calculate_formulation_returns(monthly_df)
    formulation_df.to_csv(PATH_EXPORT_FORMULATION_RET)

    # get ranking
    ranked_df, winner_df, looser_df = get_ranking_by_interval(formulation_df)        
    ranked_df.to_csv(PATH_EXPORT_RANKS)

    # # calculate holding winning
    # holding_df_winner = calculate_holding_returns(winner_df)
    # holding_df_winner.rename(columns={"holding_returns": "winning_returns"}, inplace=True)
    # holding_df_winner.to_csv(PATH_EXPORT_HOLDING_WINNERS)

    # # calculate holding returns
    # holding_df_looser = calculate_holding_returns(looser_df)
    # holding_df_looser.rename(columns={"holding_returns": "winning_returns"}, inplace=True)    
    # holding_df_looser.to_csv(PATH_EXPORT_HOLDING_LOOSERS)

    # # #
    # # combined_df = holding_df_winner.merge(holding_df_looser, on="month_startdate", suffixes=("_winner", "_looser"))
    # # combined_df.to_csv(PATH_EXPORT_HOLDING_RET)

    # for_formulation_ranking
    iteration_intervals = get_iteration_intervals(start_date, end_date, 3)

    # init dfs
    monthly_holding_df = pd.DataFrame(columns=COLUMNS_MONTHLY_HOLDING_RETURNS)
    yearly_holding_df = pd.DataFrame(columns=COLUMNS_MONTHLY_HOLDING_RETURNS)
    
    # iterate over 3 months
    for interval_start, interval_end in iteration_intervals[:10000]:
        # init vars
        year, month = interval_start.year, interval_start.month
        winner_returns = 0
        looser_returns = 0

        #
        winners_list = get_winner_looser_list(winner_df, interval_start, "winners")

        if len(winners_list) != 0:
            winner_returns = get_holding_avg_returns(monthly_df, winners_list, interval_start)

        #
        loosers_list = get_winner_looser_list(looser_df, interval_start, "loosers")

        if len(loosers_list) != 0:
            looser_returns = get_holding_avg_returns(monthly_df, loosers_list, interval_start)

        row_df = pd.DataFrame([[year, month, winner_returns, looser_returns, winners_list, loosers_list]], columns=COLUMNS_MONTHLY_HOLDING_RETURNS)

        #
        monthly_holding_df = pd.concat(
            [monthly_holding_df, row_df],
            ignore_index = True
        )

    #
    monthly_holding_df.to_csv(PATH_EXPORT_HOLDING_RET)

    # break
    # update portfolio: stock-wise capital allocation
    # print(rankings.head())

In [716]:
# run monthly simulation using market return stats
run_simulation(DATE_SIMULATION_START, DATE_SIMULATION_END)

calc
calc
calc
calc
calc
calc


C:\Users\SHAWAI~1\AppData\Local\Temp/ipykernel_8660/2570200036.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winner_df["winners"] = winner_df.groupby(["month_startdate"]).tic.transform(lambda x: ",".join(x.astype(str)))
C:\Users\SHAWAI~1\AppData\Local\Temp/ipykernel_8660/2570200036.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  looser_df["loosers"] = looser_df.groupby(["month_startdate"]).tic.transform(lambda x: ",".join(x.astype(str)))
